In [3]:
using Printf, CSV, DataFrames, Statistics, Glob, XLSX, Dates, Plots, TimeSeries, JSON, Distributions

In [8]:
data_directory = joinpath(homedir(), "research", "GADS_Processed", "cc")
files = glob("gen_*_class_CC_rating_*_state_Texas.csv", data_directory)

264-element Vector{String}:
 "/home/emco4286/research/GADS_Pr" ⋯ 23 bytes ⋯ "ass_CC_rating_1_state_Texas.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 23 bytes ⋯ "ass_CC_rating_1_state_Texas.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 23 bytes ⋯ "ass_CC_rating_1_state_Texas.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 23 bytes ⋯ "ass_CC_rating_2_state_Texas.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 23 bytes ⋯ "ass_CC_rating_2_state_Texas.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 23 bytes ⋯ "ass_CC_rating_3_state_Texas.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 23 bytes ⋯ "ass_CC_rating_2_state_Texas.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 23 bytes ⋯ "ass_CC_rating_2_state_Texas.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 23 bytes ⋯ "ass_CC_rating_2_state_Texas.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 23 bytes ⋯ "ass_CC_rating_1_state_Texas.csv"
 ⋮
 "/home/emco4286/research/GADS_Pr" ⋯ 22 bytes ⋯ "ass_CC_rating_2_state_Texas.csv"
 "/home/emco4286/research/GADS_Pr" ⋯ 22 bytes ⋯ "ass_CC_rating_1_st

In [40]:
num_fo_vec = Any[]
foh_vec = Any[]

num_po_vec = Any[]
poh_vec = Any[]

num_mo_vec = Any[]
moh_vec = Any[]

for f in files

    df = DataFrame(CSV.File(f))

    years = round(df.x[end] - df.x[begin], Day).value/365

    if size(df)[1] < 1

        push!(foh_vec, 0)
        push!(num_fo_vec, 0)

        push!(poh_vec, 0)
        push!(num_po_vec, 0)

        push!(moh_vec, 0)
        push!(num_mo_vec, 0)

        continue
    end

    outage_start = findall(diff(df[!, :y1]) .== -1)
    outage_end = findall(diff(df[!, :y1]) .== 1)

    if length(outage_end) < length(outage_start)
        outage_start = outage_start[begin: end-1]
    elseif length(outage_end) > length(outage_start)
        outage_end = outage_end[begin + 1: end]
    end

    ttr_values = outage_end .- outage_start
    ttf_values_all_types = outage_start[begin + 1: end] - outage_end[begin:end - 1]

    outage_types = df[outage_end, :y2]
    
    fo_idx = findall(outage_types .== 2)

    num_fo_average = length(fo_idx)/years
    foh_average = sum(ttr_values[fo_idx])/years

    push!(foh_vec, foh_average)
    push!(num_fo_vec, num_fo_average)

    po_idx = findall(outage_types .== 3)
    num_po_average = length(po_idx)/years
    poh_average = sum(ttr_values[po_idx])/years

    push!(poh_vec, poh_average)
    push!(num_po_vec, num_po_average)

    mo_idx = findall(outage_types .== 4)
    
    num_mo_average = length(mo_idx)/years
    moh_average = sum(ttr_values[mo_idx])/years

    push!(moh_vec, moh_average)
    push!(num_mo_vec, num_mo_average)

end

In [41]:
@printf "Average number of FO/ year: %.3f\n" mean(num_fo_vec)
@printf "Average number of PO/ year: %.3f\n" mean(num_po_vec)
@printf "Average number of MO/ year: %.3f\n" mean(num_mo_vec)

Average number of FO/ year: 7.461
Average number of PO/ year: 1.799
Average number of MO/ year: 2.780


In [42]:
@printf "Average number of FOH/ year: %.3f hours\n" mean(foh_vec)
@printf "Average number of POH/ year: %.3f hours\n" mean(poh_vec)
@printf "Average number of MOH/ year: %.3f hours\n" mean(moh_vec)

Average number of FOH/ year: 513.343 hours
Average number of POH/ year: 634.662 hours
Average number of MOH/ year: 183.775 hours


In [43]:
@printf "Average number of hours per FO: %.3f hours\n" mean(foh_vec)/mean(num_fo_vec)
@printf "Average number of hours per PO: %.3f hours\n" mean(poh_vec)/mean(num_po_vec)
@printf "Average number of hours per MO: %.3f hours\n" mean(moh_vec)/mean(num_mo_vec)

Average number of hours per FO: 68.807 hours
Average number of hours per PO: 352.820 hours
Average number of hours per MO: 66.104 hours
